In [49]:
from bs4 import BeautifulSoup
import requests
from collections import deque

def valid_wiki_link(link):
    media_file_extensions = [".ogg","jpg"]
    if( 
        ( link == None) or
        (len(link) < 6) or
        (link[:6] != "/wiki/") or
        ( ":" in link ) or
        (link[6:6+5] == "File:") or
        (link[6:6+5] == "Help:") or
        (link[6:6+5] == "Talk:") or
        (link[6:6+7] == "Portal:") or
        (link[6:6+8] == "Special:") or
        (link[6:6+9] == "Category:") or
        (link[6:6+9] == "Template:") or
        (link[6:6+10] == "Wikipedia:") or
        (link[-4:] in media_file_extensions) or
        ( "/w/index.php" in link )
    ):
        return False
    else:
        return True

'''
    DUM BFS
'''
def crawl_to_philosophy_bfs(starting_wiki_link):
    has_been_queued = {}
    queue=deque([starting_wiki_link])
    
    pages_visited = 0 
    while ( (queue[0].lower() != "/wiki/philosophy" ) and (len(queue)>0) ):
        pages_visited += 1
        page_url = "https://en.wikipedia.org" + queue.popleft()
        html_page = requests.get( page_url ).content
        soup = BeautifulSoup( html_page, 'html.parser')
        out_links =  [ link.get("href") for link in soup.find_all('a') if valid_wiki_link(link.get("href")) ]
        for link in out_links:
            if link not in has_been_queued:
                queue.append( link )
                has_been_queued[link] = True
        
        if ( pages_visited % 50 == 0 ):
            print( "Number of pages visited: {}, {}".format( pages_visited, len(queue) ) )
    
    if ((len(queue)>0) and (queue[0]=="/wiki/philosophy")):
        print("Traversed from page '{}' to Philosophy after taking {} links.".format( starting_page_tile, pages_visited ) )
    else:
        print("We reached a dead end page, congratulations. You proved the hypothesis false!")
        
    return pages_visited

def get_valid_wiki_out_links(wiki_link):
        page_url = "https://en.wikipedia.org" + wiki_link
        html_page = requests.get( page_url ).content
        soup = BeautifulSoup( html_page, 'html.parser')
        out_links = [ link.get("href").lower() for link in soup.find_all('a') if valid_wiki_link(link.get("href")) ][::-1]
        return out_links
    
'''
    What I would like this thing to do:
    * get all the links on the page with links at the top of the page having higher priority.
    * travel through the highest priority link always until we reach philosophy
'''

def crawl_to_philosophy(starting_wiki_link):
    visited = {}
    has_been_queued = { starting_wiki_link.lower(): True }
    queue=[[starting_wiki_link]]
    
    reaching_goal_possible = (len(queue)>0)
    goal_reached =  "/wiki/philosophy" in has_been_queued #(len(queue[-1])>0) and (queue[-1][-1].lower() == "/wiki/philosophy")
    
    while ( not(goal_reached) and reaching_goal_possible ):
        wiki_link = queue[-1].pop()
        if wiki_link.lower() in visited:
            continue
        else:
            visited[wiki_link.lower()] = True
        
        print("{} {}".format(len(visited), wiki_link))
        
        _out_links = get_valid_wiki_out_links(wiki_link)[::-1] #! reversing bad for performance, but enforces intented priority
        out_links = []
        for link in _out_links:
            if link not in has_been_queued:
                out_links.append( link )
                has_been_queued[ link.lower() ] = True
        
        if (len(queue[-1]) == 0):
            queue.pop()
        if (len(out_links) > 0): 
            queue.append( out_links )
        
        reaching_goal_possible = (len(queue)>0)
        goal_reached =  "/wiki/philosophy" in has_been_queued #(len(queue[-1])>0) and (queue[-1][-1].lower() != "/wiki/philosophy")
    
    if ("/wiki/philosophy" in has_been_queued):
        print("\nPhilosphy has been found starting from page '{}' after expanding {} links.".format( starting_wiki_link, len(visited) ) )
    else:
        print("We reached a dead end page, congratulations. You proved the hypothesis false!")
        
    return len(has_been_queued)

In [51]:
# It works, could 
crawl_to_philosophy("/wiki/Dolph_Lundgren") 

1 /wiki/Dolph_Lundgren
2 /wiki/main_page
3 /wiki/wikimedia_foundation

Philosphy has been found starting from page '/wiki/Dolph_Lundgren' after expanding 3 links.


938

In [ ]:
class PathFinderGame:
    def __init__(self):
        self.starting_page_title = None
        return None
    
    '''
    !!
    '''
    def start(self):
        while ( self.starting_link == None ):
            proposed_starting_page_title = input("Enter the title of the wiki page you'd like to start from: ")
            
            if ( self.verify_wiki_page_title( proposed_starting_link ) == True ):
                self.starting_page_title = proposed_starting_link
            else:
                print("There is no wiki page that exists with that title")
        
        # dfs stuff
        
        return None
    
        
    '''
        bool : 
        
        Given a page title as a String,
        Checks whether a Wikipedia Page with that title exists  
    '''
    def verify_wiki_page_title(self, title):
        return True
        